In [21]:
import numpy as np
import h5py
import scipy
from PIL import Image
from scipy import ndimage
import matplotlib.pyplot as plt
%matplotlib inline

def loading_dataset():

    with h5py.File('datasets/training images.h5', "r") as train_dataset:
        train_set_x = np.array(train_dataset["train_set_x"][:])
        train_set_y = np.array(train_dataset["train_set_y"][:])

    with h5py.File('datasets/testing images.h5', "r") as test_dataset:
        test_set_x = np.array(test_dataset["test_set_x"][:])
        test_set_y = np.array(test_dataset["test_set_y"][:])
        classes = np.array(test_dataset["list_classes"][:])

    train_set_y = train_set_y.reshape((1, train_set_y.shape[0]))
    test_set_y = test_set_y.reshape((1, test_set_y.shape[0]))

    return train_set_x, train_set_y, test_set_x, test_set_y, classes

train_set_x, train_set_y, test_set_x, test_set_y, classes = loading_dataset()

train_m = train_set_x.shape[0]
test_m = test_set_x.shape[0]
num_px = train_set_x.shape[1]

train_x_flattend = train_set_x.reshape(train_set_x.shape[0], -1).T
test_x_flattend = test_set_x.reshape(test_set_x.shape[0], -1).T

train_x = train_x_flattend/255
test_x = test_x_flattend/255

def sigmoid(z):
    x = np.exp(-z)
    y = 1 + x
    s = 1/y
    return s


def zeroinitializing(dim):
    w = np.zeros((dim,1))
    b = 0
    assert(w.shape == (dim, 1))
    assert(isinstance(b, float) or isinstance(b, int))
    return w, b

def forward_cost_backward_propagation(w, b, X, Y):
    m = X.shape[1]
    A = sigmoid(w.T @ X + b)
    cost = -1/m * np.sum(Y * np.log(A) + (1-Y) * np.log(1-A))
    dw = (X @ (A - Y).T)/m
    db = (np.sum(A-Y))/m
    assert(dw.shape == w.shape)
    assert(db.dtype == float)
    cost_values = np.squeeze(cost)
    assert(cost_values.shape == ())
    gradients = {"dw": dw,
             "db": db}
    return gradients, cost_values

def updation(w, b, X, Y, num_iterations, learning_rate, print_cost = False):
    costs = []
    for i in range(num_iterations):
        gradients, cost_values = forward_cost_backward_propagation(w,b,X,Y)
        dw = gradients["dw"]
        db = gradients["db"]
        w = w - learning_rate * dw
        b = b - learning_rate * db
        if i % 100 == 0:
            costs.append(cost_values)
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost_values))  
    parameters = {"w": w,
              "b": b} 
    gradients = {"dw": dw,
             "db": db}
    return parameters, gradients, costs

def predicted(w, b, X):
    m = X.shape[1]
    Y_prediction = np.zeros((1,m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(w.T @ X + b)
    for i in range(A.shape[1]):
        pass
        Y_prediction[: , i] = (A[:, i] > 0.5) * 1
    assert(Y_prediction.shape == (1, m))
    return Y_prediction

def main_calculate(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = False):
    w, b = zeroinitializing(X_train.shape[0])
    parameters, gradients, costs = updation(w, b, X_train, Y_train, num_iterations, learning_rate, print_cost)
    w = parameters["w"]
    b = parameters["b"]
    Y_prediction_test = predicted(w, b, X_test)
    Y_prediction_train = predicted(w, b, X_train)
    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100)) 
    final_dict = {"costs": costs,
         "Y_prediction_test": Y_prediction_test, 
         "Y_prediction_train" : Y_prediction_train, 
         "w" : w, 
         "b" : b,
         "learning_rate" : learning_rate,
         "num_iterations": num_iterations} 
    return final_dict

final_dict = main_calculate(train_x, train_set_y, test_x, test_set_y, num_iterations = 1500, learning_rate = 0.005, print_cost = True)



Cost after iteration 0: 0.693147
Cost after iteration 100: 0.584508
Cost after iteration 200: 0.466949
Cost after iteration 300: 0.376007
Cost after iteration 400: 0.331463
Cost after iteration 500: 0.303273
Cost after iteration 600: 0.279880
Cost after iteration 700: 0.260042
Cost after iteration 800: 0.242941
Cost after iteration 900: 0.228004
Cost after iteration 1000: 0.214820
Cost after iteration 1100: 0.203078
Cost after iteration 1200: 0.192544
Cost after iteration 1300: 0.183033
Cost after iteration 1400: 0.174399
train accuracy: 97.60765550239235 %
test accuracy: 70.0 %
